In [1]:
import eventstudy as es
import numpy as np
import matplotlib.pyplot as plt
import datetime
import yfinance as yf
import pandas as pd

检测输入数据

In [10]:
BSE = yf.download('MME=F', '2022-01-01', '2022-08-22')['Close']
ITC = yf.download('^BSESN', '2022-01-01', '2022-08-22')['Close']
print(type(BSE))
BSE

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
<class 'pandas.core.series.Series'>


Date
2022-01-03    1235.199951
2022-01-04    1230.400024
2022-01-05    1210.800049
2022-01-06    1216.599976
2022-01-07    1228.000000
                 ...     
2022-08-15    1019.000000
2022-08-16    1019.599976
2022-08-17    1015.099976
2022-08-18    1008.400024
2022-08-19     994.900024
Name: Close, Length: 159, dtype: float64

In [9]:
# to calcualte the return fo teh ITC and BSE sensex
ITC = ITC.pct_change()
BSE = BSE.pct_change()
ITC= pd.DataFrame(ITC)
BSE = pd.DataFrame(BSE)
BSE =BSE.dropna(axis=0)
ITC =ITC.dropna(axis=0)

# To have equal counts in ITC & BSE
df = ITC['Close'],BSE['Close']
df1 = pd.DataFrame(df)
df2 = df1.T
df2.columns = ['ITC','BSE']
df3 = df2.dropna()
df3

               Close
Date                
2022-01-04       NaN
2022-01-05  3.099320
2022-01-06 -1.300705
2022-01-07  0.956176
2022-01-10 -1.008688
...              ...
2022-08-15 -1.513051
2022-08-16 -1.107728
2022-08-17 -8.495892
2022-08-18  0.495478
2022-08-19  1.028328

[158 rows x 1 columns]


,ITC,BSE
Date,,
2022-01-05,-0.460256,3.099320
2022-01-06,-2.681614,-1.300705
2022-01-07,-1.232247,0.956176
2022-01-10,3.547516,-1.008688
2022-01-11,-0.663774,-281.091179
...,...,...
2022-08-12,-0.749571,1.623506
2022-08-16,1.908280,-1.107728
2022-08-17,0.094454,-8.495892


模拟输入数据

In [4]:
# es.Single.import_returns('returns.csv')
# es.Single.import_FamaFrench('famafrench.csv')

In [5]:
# event = es.Single.FamaFrench_3factor(
#     security_ticker = 'AAPL',
#     event_date = np.datetime64('2013-03-04'),
#     event_window = (-2,+10),
#     estimation_size = 300,
#     buffer_size = 30
# )

In [6]:
# event.plot(AR=True)
# plt.show()